# Análise de Dados

## Importando Pacotes

In [1]:
import pandas as pd
import datetime
from scipy.stats import norm
import math
import numpy as np
import datetime as dt

## Tasy (HTML + DELPH)

### Criando base de Consumo

#### Importando bases

In [2]:
mov_tasy = pd.read_csv('Movimento_HTML.csv', encoding = 'utf-8')

In [3]:
mov_delph = pd.read_csv('Movimento_Delph.csv', encoding = 'utf-8')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


#### Adicionando colunas para as Bases

In [4]:
mov_tasy['Base'] = 'Tasy HTML'

In [5]:
mov_delph['Base'] = 'Tasy Delph'

#### Consolidando as Bases

In [6]:
mov_tasy = mov_tasy.append(mov_delph)

#### Convertendo types das colunas

In [7]:
mov_tasy.drop('NR_DOCUMENTO', axis = 'columns', inplace = True)

In [8]:
mov_tasy.DT_MOVIMENTO_ESTOQUE = pd.to_datetime(mov_tasy.DT_MOVIMENTO_ESTOQUE,dayfirst = True)

In [9]:
mov_tasy['QT_MOVIMENTO'] = mov_tasy['QT_MOVIMENTO'].astype('str')

In [10]:
mov_tasy['QT_MOVIMENTO'] = mov_tasy['QT_MOVIMENTO'].str.replace(',','.')

In [11]:
mov_tasy['QT_MOVIMENTO'] = pd.to_numeric(mov_tasy['QT_MOVIMENTO'])

In [12]:
mov_tasy['VALOR_MOVIMENTO'] = mov_tasy['VALOR_MOVIMENTO'].str.replace(',','.')

In [13]:
mov_tasy['VALOR_MOVIMENTO'] = pd.to_numeric(mov_tasy['VALOR_MOVIMENTO'])

In [14]:
mov_tasy['VALOR_MOVIMENTO'].fillna(0, inplace = True)

In [15]:
mov_tasy['QT_MOVIMENTO'].fillna(0, inplace = True)

In [16]:
mov_tasy.fillna('N/A', inplace = True)

#### Transformando coluna de CD_ACAO

In [17]:
def CD_ACAO(dado):
    if dado == 1:
        return 1
    else:
        return -1

In [18]:
mov_tasy['QT_MOVIMENTO'] = mov_tasy['CD_ACAO'].apply(CD_ACAO)*mov_tasy['QT_MOVIMENTO']

In [19]:
mov_tasy['VALOR_MOVIMENTO'] = mov_tasy['CD_ACAO'].apply(CD_ACAO)*mov_tasy['VALOR_MOVIMENTO']

#### Colocando devolução de consumo 

In [20]:
def Dev_Consumo(dado):
    if dado == 'Devolução Consumo':
        return -1
    else:
        return 1

In [21]:
mov_tasy['QT_MOVIMENTO'] = mov_tasy['DS_OPERACAO'].apply(Dev_Consumo)*mov_tasy['QT_MOVIMENTO']

In [22]:
mov_tasy['VALOR_MOVIMENTO'] = mov_tasy['DS_OPERACAO'].apply(Dev_Consumo)*mov_tasy['VALOR_MOVIMENTO']

In [23]:
# Colocando meses

mov_tasy['MES'] = pd.DatetimeIndex(mov_tasy['DT_MOVIMENTO_ESTOQUE']).month

#### Base de Consumo

In [24]:
# Descrições de consumo

lista_consumo_tasy =['Execução Prescrição',
                     'Consumo',
                     'CONSIGNADO - Consumo',
                     'Reservas de lentes (INOB)',
                     'Devolução Consumo',
                     'Consumo Consignado',
                     'Reserva de OPME']

In [25]:
# Filtro últimos 90 dias

today = datetime.date(2021, 3, 2)
filter_today = str(today)
last_ninety_days = today - dt.timedelta(days=90)
filter_days = str(last_ninety_days)

In [26]:
base = today
numdays = 90
date_list = [base - datetime.timedelta(days=x) for x in range(numdays)]
ultimas_datas_tasy = pd.DataFrame(date_list)

In [27]:
# Ultimas datas

ultimas_datas_tasy.columns = ['DT_MOVIMENTO_ESTOQUE']
ultimas_datas_tasy.DT_MOVIMENTO_ESTOQUE = pd.to_datetime(ultimas_datas_tasy.DT_MOVIMENTO_ESTOQUE)
ultimas_datas_tasy.sort_values(by = 'DT_MOVIMENTO_ESTOQUE', ascending = True, inplace = True)

In [28]:
# Filtrando e criando a base

consumo_tasy = mov_tasy[mov_tasy.DS_OPERACAO.isin(lista_consumo_tasy)]
consumo_tasy = consumo_tasy[consumo_tasy['DT_MOVIMENTO_ESTOQUE'] >= filter_days]
consumo_tasy = consumo_tasy[consumo_tasy['DT_MOVIMENTO_ESTOQUE'] < filter_today]

In [29]:
# Criando Lista de Materiais

Colunas_filtro = ['Base',
                  'NM_FANTASIA_ESTAB',
                  'CD_MATERIAL',
                  'DS_REDUZIDA',
                  'FAMILIA_PADRAO',
                  'CLASSE_MATERIAL',
                  'SUBGRUPO',
                  'GRUPO']
lista_materiais = consumo_tasy[Colunas_filtro].copy()
lista_materiais.drop_duplicates(inplace=True)

In [30]:
# Construindo base_consumo_tasy

lista_materiais['Aux'] = 'Auxiliar'
ultimas_datas_tasy['Aux'] = 'Auxiliar'

base_consumo_tasy = lista_materiais.merge(ultimas_datas_tasy, how = 'outer', on=['Aux'])
base_consumo_tasy.drop(['Aux'],axis = 1,inplace=True)
base_consumo_tasy.fillna(0, inplace=True)

In [31]:
# Filtro HCLOE dezembro

base_consumo_tasy['Month'] = pd.DatetimeIndex(base_consumo_tasy['DT_MOVIMENTO_ESTOQUE']).month

In [32]:
lista_SP = ['HCLOE - Itapeva 01-21',
            'VISCLIN - Matriz Itaim - 01-03',
            'VISCLIN - Filial Alphaville -  02-94',
            'HCLOE - Vila Gomes (Tatuapé) 05-55',
            'HCLOE - Vila São Francisco (Morumbi) - 07-17',
            'SPEC - Perdizes',
            'VISCLIN - Filial Vila Olimpia - 03-75']
filtro_1 = base_consumo_tasy.NM_FANTASIA_ESTAB.isin(lista_SP)
filtro_2 = base_consumo_tasy.Month == 12
filtro = filtro_1 & filtro_2
base_consumo_tasy.drop(base_consumo_tasy[filtro].index, inplace = True)

In [33]:
# Consumo diário
consumo_diario = consumo_tasy.pivot_table(index = ['Base',
                                                   'NM_FANTASIA_ESTAB',
                                                   'CD_MATERIAL',
                                                   'DS_REDUZIDA',
                                                   'FAMILIA_PADRAO',
                                                   'CLASSE_MATERIAL',
                                                   'SUBGRUPO',
                                                   'GRUPO',
                                                   'DT_MOVIMENTO_ESTOQUE'],
                                          values = ['QT_MOVIMENTO',
                                                   'VALOR_MOVIMENTO'],
                                          aggfunc = {'QT_MOVIMENTO': sum,
                                                     'VALOR_MOVIMENTO': sum})
consumo_diario = consumo_diario.reset_index()

In [34]:
# finalizando
base_consumo_tasy = base_consumo_tasy.merge(consumo_diario,
                                            how = 'left',
                                            on = ['Base',
                                                   'NM_FANTASIA_ESTAB',
                                                   'CD_MATERIAL',
                                                   'DS_REDUZIDA',
                                                   'FAMILIA_PADRAO',
                                                   'CLASSE_MATERIAL',
                                                   'SUBGRUPO',
                                                   'GRUPO',
                                                   'DT_MOVIMENTO_ESTOQUE'])

#### Encontrando Média diária e Desvio Padrão 

In [35]:
base_consumo_tasy.fillna(0, inplace=True)

In [36]:
base_avg_std_tasy = base_consumo_tasy.pivot_table(index = ['Base',
                                                           'NM_FANTASIA_ESTAB',
                                                           'CD_MATERIAL',
                                                           'DS_REDUZIDA',
                                                           'FAMILIA_PADRAO',
                                                           'CLASSE_MATERIAL',
                                                           'SUBGRUPO',
                                                           'GRUPO'],
                                                  values = ['QT_MOVIMENTO','VALOR_MOVIMENTO'],
                                                  aggfunc = {'QT_MOVIMENTO': [np.mean, np.std, sum],
                                                             'VALOR_MOVIMENTO': sum})

In [37]:
# Reset index

base_avg_std_tasy.reset_index(inplace=True)

In [38]:
base_avg_std_tasy.columns = ['Base',
                             'NM_FANTASIA_ESTAB',
                             'CD_MATERIAL',
                             'DS_REDUZIDA',
                             'FAMILIA_PADRAO',
                             'CLASSE_MATERIAL',
                             'SUBGRUPO',
                             'GRUPO',
                             'QT_MOV_MEAN',
                             'QT_MOV_STD',
                             'QT_MOV_SUM',
                             'VALOR_MOV_SUM']

### Cálculo de Política de estoques

#### Importando Saldo 

In [39]:
saldo_html = pd.read_csv('Saldo_HTML.csv', encoding = 'utf-8')

In [40]:
saldo_delph = pd.read_csv('Saldo_Delph.csv', encoding = 'utf-8')

#### Inserindo base 

In [41]:
saldo_html['Base'] = 'Tasy HTML'

In [42]:
saldo_delph['Base'] = 'Tasy Delph'

#### Merging ambas tabelas


In [43]:
saldo_tasy = saldo_html.append(saldo_delph)

#### Convertendo unidades 

In [44]:
saldo_tasy['QT_ESTOQUE'] = saldo_tasy['QT_ESTOQUE'].astype('str')

In [45]:
saldo_tasy['QT_ESTOQUE'] = saldo_tasy['QT_ESTOQUE'].str.replace(',','.')

In [46]:
saldo_tasy['QT_ESTOQUE'] = pd.to_numeric(saldo_tasy['QT_ESTOQUE'])

In [47]:
saldo_tasy['VL_CUSTO_MEDIO'] = saldo_tasy['VL_CUSTO_MEDIO'].astype('str')

In [48]:
saldo_tasy['VL_CUSTO_MEDIO'] = saldo_tasy['VL_CUSTO_MEDIO'].str.replace(',','.')

In [49]:
saldo_tasy['VL_CUSTO_MEDIO'] = saldo_tasy['VL_CUSTO_MEDIO'].str.replace('nan','0')

In [50]:
saldo_tasy['VL_CUSTO_MEDIO'] = pd.to_numeric(saldo_tasy['VL_CUSTO_MEDIO'])

#### Merging em base_avg_std

In [51]:
colunas = ['Base',
           'NM_FANTASIA_ESTAB',
           'CD_MATERIAL',
           'DS_REDUZIDA',
           'FAMILIA_PADRAO',
           'CLASSE_MATERIAL',
           'SUBGRUPO',
           'GRUPO']

In [52]:
saldo_tasy['VALOR_ESTOQUE_ATUAL'] = saldo_tasy['QT_ESTOQUE'] * saldo_tasy['VL_CUSTO_MEDIO']

In [53]:
saldo_cons_tasy = saldo_tasy.pivot_table(index = colunas,
                                            values = ['QT_ESTOQUE',
                                                     'VALOR_ESTOQUE_ATUAL'],
                                            aggfunc = sum)
saldo_cons_tasy.reset_index(inplace=True)

In [54]:
base_avg_std_tasy = base_avg_std_tasy.merge(saldo_cons_tasy, how ='outer', on= colunas) 

In [55]:
base_avg_std_tasy.fillna(0, inplace = True)

#### Cálculo de dias de estoques 

In [56]:
base_avg_std_tasy['DOH'] = base_avg_std_tasy ['QT_ESTOQUE'] / base_avg_std_tasy['QT_MOV_MEAN']

In [57]:
base_avg_std_tasy.replace(np.inf, "Sem consumo",inplace = True)

#### Criando curva ABC 

In [58]:
aux = base_avg_std_tasy[['NM_FANTASIA_ESTAB','CD_MATERIAL','VALOR_MOV_SUM']]

In [59]:
aux_2 = aux.pivot_table(index = ['NM_FANTASIA_ESTAB',
                                 'CD_MATERIAL'],
                        columns = ['NM_FANTASIA_ESTAB'],
                        values = 'VALOR_MOV_SUM',
                        aggfunc = sum)

aux_2 = pd.DataFrame(aux_2.to_records())
aux_2.fillna(0, inplace = True)

In [60]:
lista_estab = base_avg_std_tasy.NM_FANTASIA_ESTAB.drop_duplicates().reset_index(drop = True)

In [61]:
colunas_aux_2 = aux_2.columns.tolist()
colunas_aux_2.remove('CD_MATERIAL')

In [62]:
aux_2.sort_values(by = colunas_aux_2,ascending = False, inplace = True)

In [63]:
auxiliar_2 = []
for x in lista_estab:
    auxiliar = x + " CUMSUM"
    auxiliar_2.append(auxiliar)
    aux_2[auxiliar] = aux_2[x].cumsum()

In [64]:
aux_2['AUX CUM SUM'] = aux_2['NM_FANTASIA_ESTAB']+ " CUMSUM"

In [65]:
aux_2['CUM SUM'] = aux_2.apply(lambda row: row[row['AUX CUM SUM']], axis = 1)

In [66]:
aux = pd.DataFrame(aux.pivot_table(index = ['NM_FANTASIA_ESTAB'],
                                   values = 'VALOR_MOV_SUM',
                                   aggfunc = sum).to_records())

In [67]:
aux_2 = aux_2.merge(aux, how = 'left', on = 'NM_FANTASIA_ESTAB')

In [68]:
aux_2['PERC'] = aux_2['CUM SUM'] / aux_2['VALOR_MOV_SUM']

In [69]:
def ABC_segmentation(perc):
    '''
    Creates the 3 classes A, B, and C based 
    on quantity percentages (A-60%, B-25%, C-15%)
    '''
    if perc > 0 and perc < 0.8:
        return 'A'
    elif perc >= 0.8 and perc < 0.95:
        return 'B'
    elif perc >= 0.95:
        return 'C'

In [70]:
aux_2['Class'] = aux_2['PERC'].apply(ABC_segmentation)

In [71]:
aux_2 = aux_2 [['NM_FANTASIA_ESTAB','CD_MATERIAL', 'Class']]

In [72]:
base_avg_std_tasy = base_avg_std_tasy.merge(aux_2,  how = 'left',on = ['NM_FANTASIA_ESTAB','CD_MATERIAL'])

#### Criando Estoque de Segurança

In [73]:
nvl_serv = dict({'A': 0.99, 'B':0.95,'C': 0.90})

In [74]:
base_avg_std_tasy['NVL_SERV'] = base_avg_std_tasy.Class.map(nvl_serv)

In [75]:
base_avg_std_tasy['Z'] = norm.ppf(base_avg_std_tasy.NVL_SERV)

In [76]:
# LEAD TIME ---> AQUI IRÁ MUDAR

lead_time = 7
std_lt = math.sqrt(3)

In [77]:
base_avg_std_tasy['safety_stock'] = base_avg_std_tasy['Z'] * np.sqrt(
    (((base_avg_std_tasy['QT_MOV_STD']**2)*lead_time))+((std_lt**2)*(base_avg_std_tasy['QT_MOV_MEAN']**2)))

In [78]:
base_avg_std_tasy.safety_stock = base_avg_std_tasy.safety_stock.round(0)

#### Criando estoque Max e médio

In [79]:
# Freq Pedido ---> AQUI IRÁ MUDAR

freq_pedido = 15

In [80]:
base_avg_std_tasy['Estoque_ciclo'] =  base_avg_std_tasy['QT_MOV_MEAN']*freq_pedido

In [81]:
base_avg_std_tasy['Estoque_Medio'] = base_avg_std_tasy['safety_stock'] + (base_avg_std_tasy['Estoque_ciclo']/2) 
base_avg_std_tasy['Estoque_Medio'] = base_avg_std_tasy['Estoque_Medio'].round(0)

In [82]:
base_avg_std_tasy['Estoque_Max'] = base_avg_std_tasy['safety_stock'] + (base_avg_std_tasy['Estoque_ciclo']) 
base_avg_std_tasy['Estoque_Max'] = base_avg_std_tasy['Estoque_Max'].round(0)

In [83]:
base_avg_std_tasy['Pto_Pedido'] =  (base_avg_std_tasy['QT_MOV_MEAN']*lead_time) + base_avg_std_tasy['safety_stock']
base_avg_std_tasy['Pto_Pedido'] = base_avg_std_tasy['Pto_Pedido'].round(0)

### Calculando Stock-outs

## SMART

### Criando bases de consumo 

#### Importando bases

In [84]:
# Base IOF

mov_IOF = pd.read_csv('Movimento_IOF.csv', encoding = 'utf-8')

In [85]:
# Base Hols

mov_Hosl = pd.read_csv('Movimento_Hosl.csv', encoding = 'utf-8')

In [86]:
# Base Oftalmoclin

mov_Oftalmoclin = pd.read_csv('Movimento_Oftalmoclin.csv', encoding = 'utf-8')

In [87]:
# Base Itabuna

mov_Itabuna = pd.read_csv('Movimento_Itabuna.csv', encoding = 'utf-8')

In [88]:
# Base Itaigara

mov_Itaigara = pd.read_csv('Movimento_Itaigara.csv', encoding = 'utf-8')

#### Colocando coluna de base 

In [89]:
mov_IOF['Base'] = 'IOF'

In [90]:
mov_Hosl['Base'] = 'Hosl'

In [91]:
mov_Oftalmoclin['Base'] = 'Oftalmoclin'

In [92]:
mov_Itabuna['Base'] = 'Itabuna'

In [93]:
mov_Itaigara['Base'] = 'Itaigara'

#### Corrigindo informações iniciais

In [94]:
mov_Oftalmoclin['str_nome'].fillna('OFT BARRIS', inplace = True)

In [95]:
mov_Itaigara.str_nome.fillna('DayHORC - HBA', inplace = True)

In [96]:
# Consertando tmm_descr em Itabuna

de_para = pd.DataFrame(mov_IOF[['MMA_TIPO_OPERACAO','tmm_descr', 'tmm_tipo']].drop_duplicates()).copy()
mov_Itabuna = mov_Itabuna.merge(de_para, how = 'left', on='MMA_TIPO_OPERACAO')

In [97]:
mov_Itabuna = mov_Itabuna[['MMA_TIPO_OPERACAO',
                           'mma_data_mov',
                           'MMA_SBA_COD',
                           'sba_nome',
                           'str_nome',
                           'mma_mat_cod',
                           'mma_ism_seq',
                           'MMA_ISM_SEQ',
                           'mma_lot_num',
                           'gmm_nome',
                           'fne_nome_fantasia',
                           'mat_desc_resumida',                           
                           'tmm_descr_y',
                           'tmm_tipo_y',
                           'mma_qtd',
                           'mma_valor',
                           'Base']]

In [98]:
mov_Itabuna.columns = ['MMA_TIPO_OPERACAO',
                       'mma_data_mov',
                       'MMA_SBA_COD',
                       'sba_nome',
                       'str_nome',
                       'mma_mat_cod',
                       'mma_ism_seq',
                       'MMA_ISM_SEQ',
                       'mma_lot_num',
                       'gmm_nome',
                       'fne_nome_fantasia',
                       'mat_desc_resumida',
                       'tmm_descr',
                       'tmm_tipo',
                       'mma_qtd',
                       'mma_valor',
                       'Base']

#### Unindo bases 

In [99]:
mov_smart = mov_IOF.append(mov_Hosl).append(mov_Oftalmoclin).append(mov_Itabuna).append(mov_Itaigara)

#### Manipulação base de dados 

In [100]:
mov_smart.mma_data_mov = pd.to_datetime(mov_smart.mma_data_mov).dt.normalize()

In [101]:
mov_smart['mma_qtd'] = mov_smart['mma_qtd'].astype('str')

In [102]:
mov_smart['mma_qtd'] = mov_smart['mma_qtd'].str.replace(',','.')

In [103]:
mov_smart['mma_qtd'] = pd.to_numeric(mov_smart['mma_qtd'])

In [104]:
mov_smart['mma_valor'] = mov_smart['mma_valor'].str.replace(',','.')

In [105]:
mov_smart['mma_valor'] = pd.to_numeric(mov_smart['mma_valor'])

In [106]:
mov_smart['mma_qtd'].fillna(0, inplace = True)
mov_smart['mma_valor'].fillna(0, inplace = True)

In [107]:
mov_smart.fillna('N/A', inplace = True)

#### Colocando devolução de consumo

In [108]:
def Dev_Consumo_2(dado):
    if dado == 'Entrada por Estorno de Aplica+Æo':
        return -1
    elif dado == 'Entrada por Estorno de Aplicacao':
        return -1
    elif  dado == 'Entrada por Estorno de Aplicação':
        return -1
    else:
        return 1

In [109]:
mov_smart['mma_qtd'] = mov_smart['tmm_descr'].apply(Dev_Consumo_2)*mov_smart['mma_qtd']

In [110]:
mov_smart['mma_valor'] = mov_smart['tmm_descr'].apply(Dev_Consumo_2)*mov_smart['mma_valor']

#### Base de Consumo

In [111]:
# Descrições de consumo

lista_consumo_smart =['Entrada por Estorno de Aplica+Æo',
                      'Entrada por Estorno de Aplicacao',
                      'Entrada por Estorno de Aplicação',
                      'Sa¡da de Estoque para Aplica+Æo (Consumo de Paciente)',
                      'Sa¡da de Estoque para Aplica+Æo (Consumo Interno)',
                      'Sa¡da de Estoque para Aplicação (Consumo Interno)',
                      'Saida de Estoque para Aplicacao (Consumo Interno)',
                      'Saida de Estoque para Aplicacao (Consumo de Paciente)',
                      'Saída de Estoque para Aplicação(Consumo Interno)',
                      'Saída de Estoque para Aplicação (Consumo de Paciente)',
                     'Sa¡da por Transforma+Æo']                   
                     

In [112]:
# Filtro últimos 90 dias

today = datetime.date(2021, 3, 2)
filter_today = str(today)
last_ninety_days = today - dt.timedelta(days=90)
filter_days = str(last_ninety_days)

In [113]:
# Filtrando e criando a base

consumo_smart = mov_smart[mov_smart.tmm_descr.isin(lista_consumo_smart)]
consumo_smart = consumo_smart[consumo_smart['mma_data_mov'] >= filter_days]
consumo_smart = consumo_smart[consumo_smart['mma_data_mov'] < filter_today]

In [114]:
base = today
numdays = 90
date_list = [base - datetime.timedelta(days=x) for x in range(numdays)]
ultimas_datas_smart = pd.DataFrame(date_list)

In [115]:
# Ultimas datas

ultimas_datas_smart.columns = ['mma_data_mov']
ultimas_datas_smart.mma_data_mov = pd.to_datetime(ultimas_datas_smart.mma_data_mov)
ultimas_datas_smart.sort_values(by = 'mma_data_mov', ascending = True, inplace = True)

In [116]:
# Criando Lista de Materiais

Colunas_filtro = ['Base',
                  'str_nome',
                  'mma_mat_cod',
                  'mat_desc_resumida',
                  'gmm_nome']
lista_materiais = consumo_smart[Colunas_filtro].copy()
lista_materiais.drop_duplicates(inplace=True)

In [117]:
# Construindo base_consumo_smart

lista_materiais['Aux'] = 'Auxiliar'
ultimas_datas_smart['Aux'] = 'Auxiliar'

base_consumo_smart = lista_materiais.merge(ultimas_datas_smart, how = 'outer', on=['Aux'])
base_consumo_smart.drop(['Aux'],axis = 1,inplace=True)
base_consumo_smart.fillna(0, inplace=True)

In [118]:
# Consumo diário
consumo_diario_smart = consumo_smart.pivot_table(index = ['Base',
                                                          'str_nome',
                                                          'mma_mat_cod',
                                                          'mat_desc_resumida',
                                                          'gmm_nome',
                                                          'mma_data_mov'],
                                          values = ['mma_qtd',
                                                   'mma_valor'],
                                          aggfunc = {'mma_qtd': sum,
                                                     'mma_valor': sum})
consumo_diario_smart = consumo_diario_smart.reset_index()

In [119]:
# finalizando
base_consumo_smart = base_consumo_smart.merge(consumo_diario_smart,
                                                how = 'left',
                                                on = ['Base',
                                                      'str_nome',
                                                      'mma_mat_cod',
                                                      'mat_desc_resumida',
                                                      'gmm_nome',
                                                      'mma_data_mov'])

#### Encontrando Média diária e Desvio Padrão 

In [120]:
base_consumo_smart.fillna(0, inplace=True)

In [121]:
base_avg_std_smart = base_consumo_smart.pivot_table(index = ['Base',
                                                             'str_nome',
                                                             'mma_mat_cod',
                                                             'mat_desc_resumida',
                                                             'gmm_nome'],
                                                    values = ['mma_qtd','mma_valor'],
                                                    aggfunc = {'mma_qtd': [np.mean, np.std, sum],
                                                               'mma_valor': sum})

In [122]:
# Reset index

base_avg_std_smart.reset_index(inplace=True)

In [123]:
base_avg_std_smart.columns = ['Base',
                             'NM_FANTASIA_ESTAB',
                             'CD_MATERIAL',
                             'DS_REDUZIDA',
                             'GRUPO',
                             'QT_MOV_MEAN',
                             'QT_MOV_STD',
                             'QT_MOV_SUM',
                             'VALOR_MOV_SUM']

### Cálculo de Política de estoques

#### Importando Saldo 

In [124]:
saldo_IOF = pd.read_csv('Saldo_IOF.csv', encoding = 'utf-8')

In [125]:
saldo_Hosl = pd.read_csv('Saldo_Hosl.csv', encoding = 'utf-8')

In [126]:
saldo_Oftalmoclin = pd.read_csv('Saldo_Oftalmoclin.csv', encoding = 'utf-8')

In [127]:
saldo_Itabuna = pd.read_csv('Saldo_Itabuna.csv', encoding = 'utf-8')

In [128]:
saldo_Itaigara = pd.read_csv('Saldo_Itaigara.csv', encoding = 'utf-8')

#### Inserindo base 

In [129]:
saldo_IOF['Base'] = 'IOF'

In [130]:
saldo_Hosl['Base'] = 'Hosl'

In [131]:
saldo_Oftalmoclin['Base'] = 'Oftalmoclin'

In [132]:
saldo_Itabuna['Base'] = 'Itabuna'

In [133]:
saldo_Itaigara['Base'] = 'Itaigara'

In [134]:
saldo_Oftalmoclin['str_nome'].fillna('OFT BARRIS', inplace = True)

In [135]:
saldo_Itaigara.str_nome.fillna('DayHORC - HBA', inplace = True)

#### Merging ambas tabelas


In [136]:
saldo_smart = saldo_IOF.append(saldo_Hosl).append(saldo_Oftalmoclin).append(saldo_Itabuna).append(saldo_Itaigara)

#### Convertendo unidades 

In [137]:
saldo_smart['ETQ_QUANTIDADE'] = saldo_smart['ETQ_QUANTIDADE'].astype('str')

In [138]:
saldo_smart['ETQ_QUANTIDADE'] = saldo_smart['ETQ_QUANTIDADE'].str.replace(',','.')

In [139]:
saldo_smart['ETQ_QUANTIDADE'] = pd.to_numeric(saldo_smart['ETQ_QUANTIDADE'])

In [140]:
saldo_smart['ETQ_CML_PRECO_MEDIO'] = saldo_smart['ETQ_CML_PRECO_MEDIO'].astype('str')

In [141]:
saldo_smart['ETQ_CML_PRECO_MEDIO'] = saldo_smart['ETQ_CML_PRECO_MEDIO'].str.replace(',','.')

In [142]:
saldo_smart['ETQ_CML_PRECO_MEDIO'].replace("nan",0, inplace = True)

In [143]:
saldo_smart['ETQ_CML_PRECO_MEDIO'] = pd.to_numeric(saldo_smart['ETQ_CML_PRECO_MEDIO'])

#### Merging em base_avg_std

In [144]:
colunas = ['Base',
           'str_nome',
           'ETQ_MAT_COD',
           'mat_desc_resumida',
           'gmm_nome']

In [145]:
saldo_smart['VALOR_ESTOQUE_ATUAL'] = saldo_smart.ETQ_QUANTIDADE * saldo_smart.ETQ_CML_PRECO_MEDIO

In [146]:
saldo_smart.fillna("N/A", inplace = True)

In [147]:
saldo_cons_smart = saldo_smart.pivot_table(index = colunas,
                                            values = ['ETQ_QUANTIDADE',
                                                     'VALOR_ESTOQUE_ATUAL'],
                                            aggfunc = sum)
saldo_cons_smart.reset_index(inplace=True)

In [148]:
saldo_cons_smart.VALOR_ESTOQUE_ATUAL.sum()

3752525.682283554

In [149]:
saldo_cons_smart.columns = ['Base',
                             'NM_FANTASIA_ESTAB',
                             'CD_MATERIAL',
                             'DS_REDUZIDA',
                             'GRUPO',
                            'QT_ESTOQUE',
                            'VALOR_ESTOQUE_ATUAL']

In [150]:
base_avg_std_smart = base_avg_std_smart.merge(saldo_cons_smart,
                                              how ='outer',
                                              on= ['Base',
                                                   'NM_FANTASIA_ESTAB',
                                                   'CD_MATERIAL',
                                                   'DS_REDUZIDA',
                                                   'GRUPO']) 

In [151]:
base_avg_std_smart.VALOR_ESTOQUE_ATUAL.sum()

3752525.6822835547

In [152]:
base_avg_std_smart.fillna(0, inplace = True)

#### Cálculo de dias de estoques 

In [153]:
base_avg_std_smart['DOH'] = base_avg_std_smart ['QT_ESTOQUE'] / base_avg_std_smart['QT_MOV_MEAN']

In [154]:
base_avg_std_smart.replace(np.inf, "Sem consumo",inplace = True)

#### Criando curva ABC 

In [155]:
aux = base_avg_std_smart[['NM_FANTASIA_ESTAB','CD_MATERIAL','VALOR_MOV_SUM']]

In [156]:
aux_2 = aux.pivot_table(index = ['NM_FANTASIA_ESTAB',
                                 'CD_MATERIAL'],
                        columns = ['NM_FANTASIA_ESTAB'],
                        values = 'VALOR_MOV_SUM',
                        aggfunc = sum)

aux_2 = pd.DataFrame(aux_2.to_records())
aux_2.fillna(0, inplace = True)

In [157]:
lista_estab = base_avg_std_smart.NM_FANTASIA_ESTAB.drop_duplicates().reset_index(drop = True)

In [158]:
colunas_aux_2 = aux_2.columns.tolist()
colunas_aux_2.remove('CD_MATERIAL')

In [159]:
aux_2.sort_values(by = colunas_aux_2,ascending = False, inplace = True)

In [160]:
auxiliar_2 = []
for x in lista_estab:
    auxiliar = x + " CUMSUM"
    auxiliar_2.append(auxiliar)
    aux_2[auxiliar] = aux_2[x].cumsum()

In [161]:
aux_2['AUX CUM SUM'] = aux_2['NM_FANTASIA_ESTAB']+ " CUMSUM"

In [162]:
aux_2['CUM SUM'] = aux_2.apply(lambda row: row[row['AUX CUM SUM']], axis = 1)

In [163]:
aux = pd.DataFrame(aux.pivot_table(index = ['NM_FANTASIA_ESTAB'],
                                   values = 'VALOR_MOV_SUM',
                                   aggfunc = sum).to_records())

In [164]:
aux_2 = aux_2.merge(aux, how = 'left', on = 'NM_FANTASIA_ESTAB')

In [165]:
aux_2['PERC'] = aux_2['CUM SUM'] / aux_2['VALOR_MOV_SUM']

In [166]:
def ABC_segmentation(perc):
    '''
    Creates the 3 classes A, B, and C based 
    on quantity percentages (A-60%, B-25%, C-15%)
    '''
    if perc > 0 and perc < 0.8:
        return 'A'
    elif perc >= 0.8 and perc < 0.95:
        return 'B'
    elif perc >= 0.95:
        return 'C'

In [167]:
aux_2['Class'] = aux_2['PERC'].apply(ABC_segmentation)

In [168]:
aux_2 = aux_2 [['NM_FANTASIA_ESTAB','CD_MATERIAL', 'Class']]

In [169]:
base_avg_std_smart = base_avg_std_smart.merge(aux_2,  how = 'left',on = ['NM_FANTASIA_ESTAB','CD_MATERIAL'])

#### Criando Estoque de Segurança

In [170]:
nvl_serv = dict({'A': 0.99, 'B':0.95,'C': 0.90})

In [171]:
base_avg_std_smart['NVL_SERV'] = base_avg_std_smart.Class.map(nvl_serv)

In [172]:
base_avg_std_smart['Z'] = norm.ppf(base_avg_std_smart.NVL_SERV)

In [173]:
# LEAD TIME ---> AQUI IRÁ MUDAR

lead_time = 7
std_lt = math.sqrt(3)

In [174]:
base_avg_std_smart['safety_stock'] = base_avg_std_smart['Z'] * np.sqrt(
    (((base_avg_std_smart['QT_MOV_STD']**2)*lead_time))+((std_lt**2)*(base_avg_std_smart['QT_MOV_MEAN']**2)))

In [175]:
base_avg_std_smart.safety_stock = base_avg_std_smart.safety_stock.round(0)

#### Criando estoque Max e médio

In [176]:
# Freq Pedido ---> AQUI IRÁ MUDAR

freq_pedido = 7

In [177]:
base_avg_std_smart['Estoque_ciclo'] =  base_avg_std_smart['QT_MOV_MEAN']*freq_pedido

In [178]:
base_avg_std_smart['Estoque_Medio'] = base_avg_std_smart['safety_stock'] + (base_avg_std_smart['Estoque_ciclo']/2) 
base_avg_std_smart['Estoque_Medio'] = base_avg_std_smart['Estoque_Medio'].round(0)

In [179]:
base_avg_std_smart['Estoque_Max'] = base_avg_std_smart['safety_stock'] + (base_avg_std_smart['Estoque_ciclo']) 
base_avg_std_smart['Estoque_Max'] = base_avg_std_smart['Estoque_Max'].round(0)

In [180]:
base_avg_std_smart['Pto_Pedido'] =  (base_avg_std_smart['QT_MOV_MEAN']*lead_time) + base_avg_std_smart['safety_stock']
base_avg_std_smart['Pto_Pedido'] = base_avg_std_smart['Pto_Pedido'].round(0)

In [181]:
# base_avg_std_smart.to_excel('Base_Excel_SMART.xlsx')

## Unindo bases

In [182]:
base_estoques = base_avg_std_tasy.append(base_avg_std_smart)

In [183]:
base_estoques.columns

Index(['Base', 'NM_FANTASIA_ESTAB', 'CD_MATERIAL', 'DS_REDUZIDA',
       'FAMILIA_PADRAO', 'CLASSE_MATERIAL', 'SUBGRUPO', 'GRUPO', 'QT_MOV_MEAN',
       'QT_MOV_STD', 'QT_MOV_SUM', 'VALOR_MOV_SUM', 'QT_ESTOQUE',
       'VALOR_ESTOQUE_ATUAL', 'DOH', 'Class', 'NVL_SERV', 'Z', 'safety_stock',
       'Estoque_ciclo', 'Estoque_Medio', 'Estoque_Max', 'Pto_Pedido'],
      dtype='object')

In [184]:
base_estoques['Preco_Uni'] = base_estoques['VALOR_ESTOQUE_ATUAL']/base_estoques['QT_ESTOQUE']
base_estoques['Preco_Uni'].fillna(0, inplace = True)
base_estoques['VALOR_EST_MEDIO'] = base_estoques['Estoque_Medio']*base_estoques['Preco_Uni']
base_estoques['VALOR_CONS_DIR'] = base_estoques['VALOR_MOV_SUM']*base_estoques['QT_MOV_MEAN']/base_estoques['QT_MOV_SUM']

In [185]:
base_estoques.to_excel('Base_Estoques.xlsx')

### Cálculo de consumo mensal

In [186]:
# Descrições de consumo

lista_consumo_tasy =['Execução Prescrição',
                     'Consumo',
                     'CONSIGNADO - Consumo',
                     'Reservas de lentes (INOB)',
                     'Devolução Consumo',
                     'Consumo Consignado',
                     'Reserva de OPME',
                     'Consumo da Produção',
                     'Produção']

In [187]:
# Colocando meses

mov_tasy['MES'] = pd.DatetimeIndex(mov_tasy['DT_MOVIMENTO_ESTOQUE']).month
mov_tasy['YEAR'] = pd.DatetimeIndex(mov_tasy['DT_MOVIMENTO_ESTOQUE']).year


In [188]:
day = datetime.date(2020, 10, 1)
filter_day = str(day)

In [189]:
# Filtrando e criando a base

consumo_tasy = mov_tasy[mov_tasy.DS_OPERACAO.isin(lista_consumo_tasy)]
consumo_tasy = consumo_tasy[consumo_tasy.DT_MOVIMENTO_ESTOQUE >= filter_day]

In [190]:
consumo_tasy = consumo_tasy.pivot_table(index = ['NM_FANTASIA_ESTAB',
                                                       'CD_MATERIAL',
                                                       'DS_REDUZIDA'],
                                              columns = ['MES'],
                                              values = ['QT_MOVIMENTO'],
                                              aggfunc = sum)

In [191]:
consumo_tasy.reset_index(inplace = True)

consumo_tasy.fillna(0, inplace = True)

In [192]:
# Descrições de consumo

lista_consumo_smart =['Entrada por Estorno de Aplica+Æo',
                      'Entrada por Estorno de Aplicacao',
                      'Entrada por Estorno de Aplicação',
                      'Sa¡da de Estoque para Aplica+Æo (Consumo de Paciente)',
                      'Sa¡da de Estoque para Aplica+Æo (Consumo Interno)',
                      'Sa¡da de Estoque para Aplicação (Consumo Interno)',
                      'Saida de Estoque para Aplicacao (Consumo Interno)',
                      'Saida de Estoque para Aplicacao (Consumo de Paciente)',
                      'Saída de Estoque para Aplicação(Consumo Interno)',
                      'Saída de Estoque para Aplicação (Consumo de Paciente)',
                     'Sa¡da por Transforma+Æo']                    
                     

In [193]:
# Colocando meses

mov_smart['MES'] = pd.DatetimeIndex(mov_smart['mma_data_mov']).month

In [194]:
day = datetime.date(2020, 10, 1)
filter_day = str(day)

In [195]:
# Filtrando e criando a base

consumo_smart = mov_smart[mov_smart.tmm_descr.isin(lista_consumo_smart)]
consumo_smart = consumo_smart[consumo_smart.mma_data_mov >= filter_day]

In [196]:
consumo_smart.columns

Index(['MMA_TIPO_OPERACAO', 'mma_data_mov', 'MMA_SBA_COD', 'sba_nome',
       'str_nome', 'mma_mat_cod', 'mma_ism_seq', 'MMA_ISM_SEQ', 'mma_lot_num',
       'gmm_nome', 'fne_nome_fantasia', 'mat_desc_resumida', 'tmm_descr',
       'tmm_tipo', 'mma_qtd', 'mma_valor', 'Base', 'MES'],
      dtype='object')

In [197]:
consumo_smart = consumo_smart.pivot_table(index = ['str_nome',
                                                       'mma_mat_cod',
                                                       'mat_desc_resumida'],
                                              columns = ['MES'],
                                              values = ['mma_qtd'],
                                              aggfunc = sum)

In [198]:
consumo_smart.reset_index(inplace = True)

consumo_smart.fillna(0, inplace = True)

In [199]:
consumo_smart.columns = consumo_tasy.columns

In [200]:
consumo = consumo_tasy.append(consumo_smart)

In [201]:
consumo.to_excel('CONSUMO_MENSAL.xlsx')